+ https://github.com/JuliaAcademy/DataScience/blob/master/04.%20Dimensionality%20Reduction.ipynb

In [20]:
using Pkg
Pkg.activate(".")

 Activating environment at `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\appunti\Project.toml`


In [19]:
# 3 tipi di dim red t-SNE, PCA, umap
using UMAP
using Makie
using XLSX
using VegaDatasets
using DataFrames
using MultivariateStats
using RDatasets
using StatsBase
using Statistics
using LinearAlgebra
using Plots
using ScikitLearn
using MLBase
using Distances

In [29]:

C = DataFrame(VegaDatasets.dataset("cars"))

C[1:10,1:9]

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower
,String,Float64?,Int64,Float64,Int64?
1,chevrolet chevelle malibu,18.0,8,307.0,130
2,buick skylark 320,15.0,8,350.0,165
3,plymouth satellite,18.0,8,318.0,150
4,amc rebel sst,16.0,8,304.0,150
5,ford torino,17.0,8,302.0,140
6,ford galaxie 500,15.0,8,429.0,198
7,chevrolet impala,14.0,8,454.0,220
8,plymouth fury iii,14.0,8,440.0,215
9,pontiac catalina,14.0,8,455.0,225


In [34]:
dropmissing!(C)#non vogliamo gestire valori missing -> li togliamo
M = Matrix(C[:,2:7]) #funzioni riduzione dimensionalita operano su matrici quindi prendiamoci matrice
# prendiamo solo colonne per valori "Miles_per_Gallon" "Cylinders" "Displacement" "Horsepower" "Weight_in_lbs" "Acceleration"
names(C) # vediamo cosa contiene

9-element Array{String,1}:
 "Name"
 "Miles_per_Gallon"
 "Cylinders"
 "Displacement"
 "Horsepower"
 "Weight_in_lbs"
 "Acceleration"
 "Year"
 "Origin"

In [35]:
car_origin = C[:,:Origin]# prendiamo vettori origini
carmap = labelmap(car_origin) #from MLBase, mappa i valori stringa in numeri unicamente rappresentanti al valore
uniqueids = labelencode(carmap,car_origin) # prende stringa valori e li cambia con la mappatura in numero

392-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 2
 1
 1
 1
 3
 1
 1
 1

## PCA
prima centriamo i dati

In [37]:
M

392×6 Array{Float64,2}:
 18.0  8.0  307.0  130.0  3504.0  12.0
 15.0  8.0  350.0  165.0  3693.0  11.5
 18.0  8.0  318.0  150.0  3436.0  11.0
 16.0  8.0  304.0  150.0  3433.0  12.0
 17.0  8.0  302.0  140.0  3449.0  10.5
 15.0  8.0  429.0  198.0  4341.0  10.0
 14.0  8.0  454.0  220.0  4354.0   9.0
 14.0  8.0  440.0  215.0  4312.0   8.5
 14.0  8.0  455.0  225.0  4425.0  10.0
 15.0  8.0  390.0  190.0  3850.0   8.5
 15.0  8.0  383.0  170.0  3563.0  10.0
 14.0  8.0  340.0  160.0  3609.0   8.0
 15.0  8.0  400.0  150.0  3761.0   9.5
  ⋮                                ⋮
 25.0  6.0  181.0  110.0  2945.0  16.4
 38.0  6.0  262.0   85.0  3015.0  17.0
 26.0  4.0  156.0   92.0  2585.0  14.5
 22.0  6.0  232.0  112.0  2835.0  14.7
 32.0  4.0  144.0   96.0  2665.0  13.9
 36.0  4.0  135.0   84.0  2370.0  13.0
 27.0  4.0  151.0   90.0  2950.0  17.3
 27.0  4.0  140.0   86.0  2790.0  15.6
 44.0  4.0   97.0   52.0  2130.0  24.6
 32.0  4.0  135.0   84.0  2295.0  11.6
 28.0  4.0  120.0   79.0  2625.0  18.6
 31

In [38]:
# normaslizziamo i dati sottraendone la loro media e dividendo per la loro deviazione standard
data = M
data = (data .- mean(data,dims = 1))./ std(data,dims=1)

392×6 Array{Float64,2}:
 -0.697747   1.48205    1.07591    0.663285   0.619748   -1.28362
 -1.08212    1.48205    1.48683    1.57258    0.842258   -1.46485
 -0.697747   1.48205    1.18103    1.18288    0.539692   -1.64609
 -0.953992   1.48205    1.04725    1.18288    0.53616    -1.28362
 -0.82587    1.48205    1.02813    0.923085   0.554997   -1.82732
 -1.08212    1.48205    2.24177    2.42992    1.60515    -2.00855
 -1.21024    1.48205    2.48068    3.00148    1.62045    -2.37102
 -1.21024    1.48205    2.34689    2.87158    1.57101    -2.55226
 -1.21024    1.48205    2.49023    3.13138    1.70404    -2.00855
 -1.08212    1.48205    1.86908    2.22208    1.02709    -2.55226
 -1.08212    1.48205    1.80219    1.70248    0.689209   -2.00855
 -1.21024    1.48205    1.39127    1.44268    0.743365   -2.73349
 -1.08212    1.48205    1.96464    1.18288    0.922314   -2.18979
  ⋮                                                       ⋮
  0.199113   0.309571  -0.128168   0.143685  -0.0383613   

In [39]:
# per la funzione PCA un osservazione è rappresentata come una colonna,
#dobbiamo quindi passargli trasposizione righe/colonne poichè per noi una colonna è una variabile
p = fit(PCA,data',maxoutdim=2) #prima fittiamo il modello tramite PCA
# maxoputdim è 2 perchè vogliamo layout a 2 dimensioni

PCA(indim = 6, outdim = 2, principalratio = 0.9194828785333573)

In [ ]:
# otteniamo matrice proiez con comando projection
P = projection(p)